# Evolver Loop 1 Analysis

Analyze the current state and identify opportunities for improvement.

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
import os

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def create_tree_polygon(x, y, deg):
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    vertices = []
    for tx, ty in zip(TX, TY):
        rx = tx * cos_a - ty * sin_a + x
        ry = tx * sin_a + ty * cos_a + y
        vertices.append((rx, ry))
    return Polygon(vertices)

def compute_bounding_side(polygons):
    if not polygons:
        return 0
    all_points = []
    for poly in polygons:
        all_points.extend(list(poly.exterior.coords))
    all_points = np.array(all_points)
    min_x, min_y = all_points.min(axis=0)
    max_x, max_y = all_points.max(axis=0)
    return max(max_x - min_x, max_y - min_y)

def compute_score_for_n(df, n):
    prefix = f"{n:03d}_"
    trees = df[df['id'].str.startswith(prefix)]
    if len(trees) != n:
        return float('inf')
    polygons = []
    for _, row in trees.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        polygons.append(create_tree_polygon(x, y, deg))
    side = compute_bounding_side(polygons)
    return side**2 / n

def compute_total_score(df):
    total = 0
    for n in range(1, 201):
        total += compute_score_for_n(df, n)
    return total

print("Functions defined")

Functions defined


In [2]:
# Check the SmartManoj scoreboard submission
df_smartmanoj = pd.read_csv('/home/code/external_data/smartmanoj_submission.csv')
score_smartmanoj = compute_total_score(df_smartmanoj)
print(f"SmartManoj scoreboard score: {score_smartmanoj:.6f}")

# Compare with our current best
print(f"\nCurrent best: 70.659959")
print(f"SmartManoj: {score_smartmanoj:.6f}")
print(f"Difference: {score_smartmanoj - 70.659959:.6f}")

SmartManoj scoreboard score: 70.743774

Current best: 70.659959
SmartManoj: 70.743774
Difference: 0.083815


In [3]:
# Analyze per-N scores to find where improvements are possible
df_best = pd.read_csv('/home/submission/submission.csv')
df_smartmanoj = pd.read_csv('/home/code/external_data/smartmanoj_submission.csv')

per_n_scores_best = []
per_n_scores_smartmanoj = []

for n in range(1, 201):
    score_best = compute_score_for_n(df_best, n)
    score_sm = compute_score_for_n(df_smartmanoj, n)
    per_n_scores_best.append(score_best)
    per_n_scores_smartmanoj.append(score_sm)

# Find where SmartManoj is better
improvements = []
for n in range(1, 201):
    diff = per_n_scores_best[n-1] - per_n_scores_smartmanoj[n-1]
    if diff > 0.0001:
        improvements.append((n, diff, per_n_scores_best[n-1], per_n_scores_smartmanoj[n-1]))

print(f"SmartManoj is better for {len(improvements)} N values:")
for n, diff, best, sm in sorted(improvements, key=lambda x: -x[1])[:20]:
    print(f"  N={n}: diff={diff:.6f} (best={best:.6f}, sm={sm:.6f})")

SmartManoj is better for 0 N values:


In [4]:
# Create ensemble from both sources
ensemble_rows = []
total_ensemble_score = 0

for n in range(1, 201):
    score_best = per_n_scores_best[n-1]
    score_sm = per_n_scores_smartmanoj[n-1]
    
    if score_sm < score_best:
        # Use SmartManoj
        prefix = f"{n:03d}_"
        trees = df_smartmanoj[df_smartmanoj['id'].str.startswith(prefix)].copy()
        ensemble_rows.append(trees)
        total_ensemble_score += score_sm
    else:
        # Use current best
        prefix = f"{n:03d}_"
        trees = df_best[df_best['id'].str.startswith(prefix)].copy()
        ensemble_rows.append(trees)
        total_ensemble_score += score_best

print(f"Ensemble score: {total_ensemble_score:.6f}")
print(f"Improvement over current best: {70.659959 - total_ensemble_score:.6f}")

Ensemble score: 70.659959
Improvement over current best: -0.000000


In [ ]:
# Save the ensemble if it's better
if total_ensemble_score < 70.659959:
    ensemble_df = pd.concat(ensemble_rows, ignore_index=True)
    ensemble_df.to_csv('/home/submission/submission.csv', index=False)
    print(f"Saved improved ensemble with score {total_ensemble_score:.6f}")
else:
    print("No improvement from ensemble")

In [5]:
# Analyze per-N score distribution
import matplotlib.pyplot as plt

# Calculate per-N scores
per_n_scores = []
per_n_sides = []
for n in range(1, 201):
    score = per_n_scores_best[n-1]
    per_n_scores.append(score)
    # side = sqrt(score * n)
    side = np.sqrt(score * n)
    per_n_sides.append(side)

# Find which N values contribute most to total score
total = sum(per_n_scores)
print(f"Total score: {total:.6f}")
print(f"\nTop 20 N values by score contribution:")
sorted_by_score = sorted(enumerate(per_n_scores, 1), key=lambda x: -x[1])
for n, score in sorted_by_score[:20]:
    pct = score / total * 100
    print(f"  N={n}: score={score:.6f} ({pct:.2f}%)")

Total score: 70.659959

Top 20 N values by score contribution:
  N=1: score=0.661250 (0.94%)
  N=2: score=0.450779 (0.64%)
  N=3: score=0.434745 (0.62%)
  N=5: score=0.416850 (0.59%)
  N=4: score=0.416545 (0.59%)
  N=7: score=0.399897 (0.57%)
  N=6: score=0.399610 (0.57%)
  N=9: score=0.387415 (0.55%)
  N=8: score=0.385407 (0.55%)
  N=15: score=0.379203 (0.54%)
  N=10: score=0.376630 (0.53%)
  N=21: score=0.376451 (0.53%)
  N=20: score=0.376057 (0.53%)
  N=11: score=0.375736 (0.53%)
  N=22: score=0.375258 (0.53%)
  N=16: score=0.374128 (0.53%)
  N=26: score=0.373997 (0.53%)
  N=12: score=0.372724 (0.53%)
  N=13: score=0.372294 (0.53%)
  N=25: score=0.372144 (0.53%)


In [ ]:
# Calculate theoretical lower bound for N=1 (single tree at optimal rotation)
# The tree has specific dimensions - let's compute the minimum bounding box

# Tree vertices
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

# Find optimal rotation for N=1
best_side = float('inf')
best_angle = 0

for angle in np.linspace(0, 360, 3600):
    rad = np.radians(angle)
    cos_a, sin_a = np.cos(rad), np.sin(rad)
    
    rotated_x = [tx * cos_a - ty * sin_a for tx, ty in zip(TX, TY)]
    rotated_y = [tx * sin_a + ty * cos_a for tx, ty in zip(TX, TY)]
    
    width = max(rotated_x) - min(rotated_x)
    height = max(rotated_y) - min(rotated_y)
    side = max(width, height)
    
    if side < best_side:
        best_side = side
        best_angle = angle

print(f"Optimal N=1 rotation: {best_angle:.2f} degrees")
print(f"Optimal N=1 side: {best_side:.6f}")
print(f"Optimal N=1 score: {best_side**2:.6f}")
print(f"Current N=1 score: {per_n_scores[0]:.6f}")
print(f"Gap: {per_n_scores[0] - best_side**2:.6f}")